# 1. 라이브러리 불러오기

In [14]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time
import re
import csv 
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

from bs4 import BeautifulSoup
from tqdm import tqdm

ModuleNotFoundError: No module named 'selenium'

# 2. 수집한 리뷰들 불러오기

## 2-(1) 블로그 리뷰 불러오기

In [3]:
#추출한 블로그 리뷰 csv파일 불러오기
blog_review_list=pd.read_csv('place_blog_review.csv',encoding='utf-8')
blog_review=blog_review_list[['num', 'name','blog or review','blog name','like_count_list','command_count_list','date_list','img_count','neighber_count','last_content_date','blog review content']]

In [4]:
blog_review.shape

(1584, 11)

In [5]:
#불러온 블로그 리뷰 중 결측값 확인
blog_review.isnull().any()

num                    False
name                   False
blog or review         False
blog name               True
like_count_list        False
command_count_list     False
date_list              False
img_count              False
neighber_count         False
last_content_date      False
blog review content     True
dtype: bool

In [6]:
#불러온 블로그 리뷰 중 결측값이 있는 행 제거
blog_review2=blog_review.dropna(axis=0)

In [7]:
#블로그 리뷰 결측값 제거 완료
blog_review2.isnull().any()

num                    False
name                   False
blog or review         False
blog name              False
like_count_list        False
command_count_list     False
date_list              False
img_count              False
neighber_count         False
last_content_date      False
blog review content    False
dtype: bool

In [8]:
blog_review2

,num,name,blog or review,blog name,like_count_list,command_count_list,date_list,img_count,neighber_count,last_content_date,blog review content
0,1,배곧 또잇카페 배곧점,블로그 리뷰,TTOEATCAFE 배곧카페 뷰맛집!,5,6,2021. 8. 21. 13:36,21,"1,071",2021-11-16,배곧 한울공원이 산책하기에 너무 좋은곳이라 자주 가요~오후시간에 산책하면 노을지는모...
1,2,배곧 또잇카페 배곧점,블로그 리뷰,배곧 한울공원 뷰 [또잇카페],7,4,2021. 11. 14. 22:39,23,191,2021-11-14,"안녕 여러분 ~~주말에 배곧 한울공원에 갔다가뷰맛집 , 케이쿠 맛집이라는또잇카페를 ..."
2,3,배곧 또잇카페 배곧점,블로그 리뷰,배곧카페 TTOEATCAFE 또잇 : 배곧한울공원카페 / 배곧디저트,5,2,2021. 11. 15. 8:39,30,111,2021-11-15,주말에 다녀오기 딱 좋은 카페를 발견했다!생명공원과 한울공원 가까이에 있는 배곧디저...
3,4,배곧 또잇카페 배곧점,블로그 리뷰,"시흥 배곧카페 또잇카페, 정왕동 신상 디저트 카페",22,18,2021. 11. 11. 22:01,22,547,2021-11-16,요즘 새로 생긴 신상 카페 투어중인 친구와 저는오랜만에 배곧에서 점심을 먹고 디저트...
4,5,배곧 또잇카페 배곧점,블로그 리뷰,시흥 배곧 카페 추천: 또잇카페,7,1,2021. 10. 26. 13:19,16,311,2021-11-16,시댁갔다 오빠랑 어머님이랑 셋이 방문한 배곧 카페오빠한테 폭풍잔소리한 덕분인지 예쁜...
...,...,...,...,...,...,...,...,...,...,...,...
1579,77,배곧 원조부안집 배곧점,블로그 리뷰,배곧 맛집 원조부안집 배곧점,1,2,2021. 1. 5. 16:59,6,2,2021-2-2,"배곧 맛집 원조부안집 배곧점#배곧맛집, #배곧고기집, #배곧고깃집, #배곧삼겹살, ..."
1580,78,배곧 원조부안집 배곧점,블로그 리뷰,시흥맛집 *원조부안집 배곧점*,1,1,2020. 8. 16. 20:05,9,11,2021-10-24,원조부안집 배곧점 고고https://vo.la/yNXQ오늘은 시흥맛집 원조부안집 배...
1581,79,배곧 원조부안집 배곧점,블로그 리뷰,[시흥 맛집] 부안오득살이 끝내주는 '원조부안집 배곧점',2,2,2020. 6. 30. 21:10,6,36,2021-6-3,안녕하세요:D은별이 입니다!!이번 포스팅에서는 시흥 맛집'원조부안집 배곧점'을 소개...
1582,80,배곧 원조부안집 배곧점,블로그 리뷰,"원조 부안 집 배곧점, 시흥 맛집제목없음",4,2,2020. 8. 14. 15:18,6,180,2021-10-31,안녕하세요!!반갑습니다!!네이버 불러 그 지 산입니다!!오랜 장마에 혹우 경보로 ...


## 2-(2) 방문자 리뷰 불러오기

In [9]:
#추출한 방문자리뷰 csv파일 불러오기
visitor_review_list=pd.read_csv('place_visitor_review.csv',encoding='utf-8')
visitor_review=visitor_review_list[['num','name','blog or review','visitor review content']]

In [10]:
#불러온 방문자 리뷰 중 결측값 확인
visitor_review.isnull().any()

num                       False
name                      False
blog or review            False
visitor review content     True
dtype: bool

In [11]:
#불러온 방문자 리뷰 중 결측값이 있는 행 제거
visitor_review2=visitor_review.dropna(axis=0)

In [12]:
#방문자 리뷰 결측값 제거 완료
visitor_review2.isnull().any()

num                       False
name                      False
blog or review            False
visitor review content    False
dtype: bool

In [13]:
visitor_review2

,num,name,blog or review,visitor review content
0,1,배곧 또잇카페 배곧점,방문자 리뷰,굿
1,2,배곧 또잇카페 배곧점,방문자 리뷰,좋아요
2,3,배곧 또잇카페 배곧점,방문자 리뷰,조아요
3,4,배곧 또잇카페 배곧점,방문자 리뷰,굿
4,5,배곧 또잇카페 배곧점,방문자 리뷰,바다공원 앞에 위치해서 편하고 좋아요!^^ 공원에서 운동하고 아메리카노 급땡겨서 또...
...,...,...,...,...
14006,14,배곧 카페바이마스,방문자 리뷰,밀크티는 정말 진리네요👍
14007,15,배곧 카페바이마스,방문자 리뷰,원두 선택할 수 있어서 좋아요.디카페인이 있는것도 별 5개⭐️
14008,16,배곧 카페바이마스,방문자 리뷰,음료 다 맛있어요 배곧 커피맛집이에요
14009,17,배곧 카페바이마스,방문자 리뷰,커피맛있어요


# 3. 전처리(특수문자, 띄어쓰기, 불용어 제거 등)

## 3-(1) 전처리 라이브러리 호출

In [1]:
#전처리를 하기 위해 필요한 라이브러리 호출
import os
import sys
import re
import konlpy
import pandas as pd
from konlpy.tag import Kkma
from konlpy.tag import Okt
#import soynlp
import kss
from pykospacing import Spacing
from tqdm import tqdm
import time

kkma = Kkma()
okt = Okt()
spacing = Spacing()

## 3-(2) 기본 전처리

In [78]:
corpus = []
def normalization(cr_review):
    for i in tqdm(range(0, len(cr_review)), desc='전처리 진행율'):
        review = re.sub('\n','', str(cr_review[i]))
        # 기본 전처리
        review = re.sub('\u200b','',review) # 폭 없는 공백 제거
        review = re.sub('\xa0','',review) # Latin1 (ISO 8859-1)비 공백 공간 제거
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\;\!\-\,\_\~\$\'\"\:]', '',review) # 여러 기호 제거
        review = re.sub(r'[^가-힣-\s0-9]', '', review) # 한글과 숫자만 살리고 제거
        review = re.sub(r'\s+', ' ', review) # 다중 공백 제거
        review = re.sub(r'^\s+', '', review) # 시작 공백 제거
        # 표현 및 문법 전처리
        review = re.sub('라구요|라구여|라구용|라고염|라고여|라고용', '라고요', review)
        review = re.sub('어용|어염|어여', '어요', review)
        review = re.sub('봐용|봐염|봐여', '봐요', review)
        review = re.sub('게용|게염|게여', '게요', review)
        review = re.sub('했당', '했다', review)
        review = re.sub('았당', '았다', review)
        review = re.sub('에용|에염|에여|예염', '에요', review)
        review = re.sub('세용|세염|세여', '세요', review)
        review = re.sub('께용|께염|께여|께유', '께요', review)
        review = re.sub('해용|해염|해여|해유', '해요', review)
        review = re.sub('네용|네염|네여|네유', '네요', review)
        review = re.sub('아용|아염|아여', '아요', review)
        review = re.sub('니당', '니다', review)
        review = re.sub('괜춘', '괜찮네요', review)
        review = re.sub('이뻐', '예뻐', review)
        review = re.sub('이쁘', '예쁘', review)
        review = re.sub('이쁜', '예쁜', review)
        review = re.sub('고기집', '고깃집', review)
        review = re.sub('같아용|같아염|같아여', '같아요', review)
        review = re.sub('같네용|같네염|같네여', '같네요', review)
        review = re.sub('이구용', '이구요', review)
        review = re.sub('었 따', '었다', review)
        # 띄어쓰기
        review = spacing(review)
        # 띄어쓰기 전처리
        review = re.sub('니\s다', '니다', review)
        review = re.sub('라\s고요|라고\s요', '라고요', review)
        review = re.sub('배\s곧', '배곧', review)
        review = re.sub('또\s잇', '또잇', review)
        review = re.sub('와\s규', '와규', review)
        review = re.sub('에\s비야', '에비야', review)
        #문장 분절
        review = kss.split_sentences(review) 

        for i in range(len(review)):
            sr = ''
            sr += review[i]+'.'
            sr = re.sub(r'[^가-힣-\s0-9\.]', '', sr)
            corpus.append(sr)
            
        
    return corpus

## 3-(1) 블로그 리뷰 전처리 및 csv파일 형태로 저장

In [39]:
def convert_blog_review(df):
    df.dropna(inplace = True) # 결측치가 있는 행 제거
    df.reset_index(inplace = True) # 인덱스 초기화
    blog_review = df['blog review content'] # 블로그 리뷰를 리스트로 꺼내서
    corpus = normalization(blog_review) # 블로그 리뷰 정규화 및 띄어쓰기 문장 분절
    df.insert(11, 'blog review', corpus) # 11번째 행에 전처리된 블로그 리뷰 삽입
    df.drop(['blog review content', 'index'], axis=1, inplace=True) #기존 리뷰 행 삭제
    df.to_csv('processed_place_blog_review1.csv', encoding='utf-8-sig') # csv 파일로 저장

In [40]:
convert_blog_review(blog_review2)

NameError: name 'blog_review2' is not defined

## 3-(3) 네이버 플레이스 방문자 리뷰 전처리

In [80]:
def convert_place_review(df):
    df.dropna(inplace=True) # 데이터프레임 결측값 제거
    df.reset_index(inplace=True) # 데이터 프레임 인덱스 초기화
    visitor_review = df['visitor review content'] # 플레이스 리뷰를 리스트로 꺼내서
    corpus = normalization(visitor_review) # 플레이스 리뷰 정규화 및 띄어쓰기 문장 분절
    df.insert(4, 'visitor review',  corpus) # 4번째 행에 전처리된 플레이스 리뷰 삽입
    df.dropna(inplace=True) # 정규화를 하면 플레이스 리뷰의 경우 이모티콘만 있는 경우에 결측값이 되기 때문에 한번 더 결측값 제거
    df.drop(['visitor review content', 'index'], axis=1, inplace=True) # 기존 리뷰 행 삭제
    df.to_csv('processed_place_visitor_review1.csv', encoding='utf-8-sig')

In [81]:
df = pd.read_csv('place_visitor_review.csv')

In [17]:
convert_place_review(df)

전처리 진행율: 100%|███████████████████████████████████████████████████████████| 14006/14006 [1:43:59<00:00,  2.24it/s]
